## A) 
Hur många länder som är med i datan?

## B)
Vilka länder är med? (förkortningarna räcker)

## C) 
Vilka sporter är med?

## D)
Vilka medaljtyper finns det?

## E)
Ta reda på statistik för åldern: medelvärde, median, min, max, standardavvikelse

## F)
Utforska datan vidare med egna frågor
- Vilka åldersintervaller tar flest medaljer?
- Varierar medelåldern mellan mästerskapen? 
- Könsfördelning mellan mästerskapen?
- Eller andra intressanta grejer...

## G)
Diagram över könsfördelningen

## H)
Diagram över topp 10 länder som tagit flest medaljer

## I) 
Plotta gärna fler saker som är intressant. *(förslagsvis från F)*